In [ ]:
import gc
import os
os.chdir("..")
os.chdir('input')
os.chdir('mypractices')
os.chdir('TwitterSentimentExtraction')

In [ ]:
import pandas as pd
import shutil
import shutil
from eda import *
from csv2json import *

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')
test_data = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/test.csv")
ted_data_for_second = test_data

In [ ]:
# preprocessing for the 1st model
test_data = prepare_data_for_first_model(test_data)

In [ ]:
# creating sample сsv2json
create_samples('/kaggle/working', test_data)

In [ ]:
# generating dataset for 1st model predictions
owd = os.getcwd()
os.chdir('generate')
!python generate_rambler2011_json_NLI_M_first_model.py
os.chdir(owd)

In [ ]:
# 1st model predictions
if os.path.exists('/kaggle/working/results'):
        shutil.rmtree('/kaggle/working/results')

!python run_classifier_TABSA_pretrained.py \
--task_name rambler2011_json_NLI_M \
--data_dir /kaggle/working/ \
--vocab_file '/kaggle/input/mypractices/TwitterSentimentExtraction/conversational_cased_L-12_H-768_A-12_v1/vocab.txt' \
--bert_config_file '/kaggle/input/mypractices/TwitterSentimentExtraction/conversational_cased_L-12_H-768_A-12_v1/bert_config.json' \
--init_checkpoint '/kaggle/input/mypractices/TwitterSentimentExtraction/conversational_cased_L-12_H-768_A-12_v1/pytorch_model.bin' \
--eval_test \
--do_lower_case \
--max_seq_length 512 \
--train_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 2.0 \
--output_dir /kaggle/working/results/ \
--seed 42 \
--model_number first_model

In [ ]:
# results of first task to submission file (0 means marker=text)
data_first_model = pd.read_csv('/kaggle/working/test_NLI_M.tsv', sep='\t')
results_first_model = []
with open('/kaggle/working/results/results.txt') as f:
    for line in f:
        results_first_model.append(int(line.split()[0]))
data_first_model['answer'] = results_first_model
data_first_model['sentence1'] = data_first_model['sentence1'].apply(lambda x: x[15:])
test_data['text_tok'] = test_data['text_tok'].apply(lambda x: x[12:])

test_data = test_data.set_index('selected_text_tok').join(data_first_model.set_index('tonal_word'))[
    ['textID', 'text', 'text_tok', 'answer']]
test_data['answer'] = test_data.apply(lambda row: row['text'] if row['answer'] == 0 else 'ask 2nd model', axis=1)
for_2nd_model = test_data[test_data['answer'] == 'ask 2nd model']['textID']

sample_submission = sample_submission.set_index('textID').join(test_data.set_index('textID'))[
    ['answer']].reset_index()

In [ ]:
gc.collect()

In [ ]:
# preprocessing for 2nd model
test_data = ted_data_for_second[ted_data_for_second['textID'].isin(for_2nd_model)]
test_data = prepare_data_for_second_model(test_data, 3)
test_data.to_csv('/kaggle/working/test_data_for_2nd_model.csv', index=False, sep='\t')

In [ ]:
# creating sample сsv2json
create_samples_2nd_model('raw_data', test_data)

In [ ]:
# generating dataset for 2nd model predictions
os.chdir('generate')
!python generate_rambler2011_json_NLI_M_second_model.py
os.chdir(owd)

In [ ]:
# 2nd model predictions
if os.path.exists('/kaggle/working/results'):
	shutil.rmtree('/kaggle/working/results')

!python run_classifier_TABSA_pretrained.py \
--task_name rambler2011_json_NLI_M \
--data_dir /kaggle/working/ \
--vocab_file '/kaggle/input/mypractices/TwitterSentimentExtraction/conversational_cased_L-12_H-768_A-12_v1/vocab.txt' \
--bert_config_file '/kaggle/input/mypractices/TwitterSentimentExtraction/conversational_cased_L-12_H-768_A-12_v1/bert_config.json' \
--init_checkpoint '/kaggle/input/mypractices/TwitterSentimentExtraction/conversational_cased_L-12_H-768_A-12_v1/pytorch_model.bin' \
--eval_test \
--do_lower_case \
--max_seq_length 512 \
--train_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 2.0 \
--output_dir /kaggle/working/results/ \
--seed 42 \
--model_number second_model

In [ ]:
#  results of second task to submission file + extract best marker for each sentence
data_second_model = pd.read_csv('/kaggle/working/test_NLI_M.tsv', sep='\t')
answer_second_model = []
logit_zero = []
logit_one = []

# with open('results/rambler2011_json/NLI_M/results_second.txt') as f:
with open('/kaggle/working/results/results.txt') as f:
    for line in f:
        answer_second_model.append(int(line.split()[0]))
        logit_zero.append(float(line.split()[1]))
        logit_one.append(float(line.split()[2]))
        
data_second_model['answer'] = answer_second_model
data_second_model['logit_zero'] = logit_zero
data_second_model['logit_one'] = logit_one
data_second_model['best'] = data_second_model.groupby(['id'])['logit_one'].rank(method='first', ascending=True)
ans = data_second_model[data_second_model['best'] == 1][['id', 'sentence2']]
ans['sentence2'] = ans['sentence2'].apply(lambda x: x[25:])
ans.rename(columns={"id": "textID", "sentence2": "selected_text"}, inplace=True)

In [ ]:
gc.collect()

In [ ]:
sample_submission.rename(columns={"textID": "textID", "answer": "selected_text"}, inplace=True)
sample_submission.set_index('textID', inplace=True)
sample_submission.update(ans.set_index('textID'))
sample_submission = sample_submission.reset_index()
sample_submission.to_csv('/kaggle/working/submission.csv', index=False)